# Advanced Python - Building Scalable Applications

## Module 2

#### Python ```threading``` module: a deep-dive
 - Python threading module API
 - Creating and managing threads.
 - An overview on threading module.
 - Using the Thread class and the Timer class.
 - Active threads Vs. Daemon threads.
 - Helper functions in the threading module.

#### Exercise: 
 - Implement a ```joinall()``` function to wait for all threads to exit

 #### Parallel processing using ```multiprocessing``` module
 - Multitasking using multiprocessing.Process
 - Process Vs Thread: performance and design implications.
 - Similarities and differences between ```Process``` and ```Thread``` class API
 - Helper functions in the ```multiprocessing``` module.

### Python ```threading``` module API

In [2]:
from threading import Thread

##### ```Thread``` constructor arguments:
``` 
   t = Thread(
         group=None,     # Not implemented. Leave it as None
         target=None,    # The function to be run as a separate thread
         name=None,      # Set the string name of a thread (defaults to Thread-1...)
         args=(),        # Positional arguments to target function
         kwargs=None,    # Keyword arguments to target function
         *,
         daemon=None     # Set to True to create a daemon thread
   )
```


#### Important ```Thread``` attributes / methods:
```
    from threading import Thread
    from time import sleep

    t = Thread(target=sleep, args=(60,))   # Create a new Thread instance for a function sleep(60)
    
    t.start()  # Starts the Thread (initializes the thread internally via OS and invokes the run() method as new Thread)
    t.join()   # Waits indefinitely until the target function spawned off as a separate thread returns

    t.is_alive()  # Returns True if Thread is alive, or False if the thread completed as the target function returned.

    t.name   # Returns the name of the Thread as a string. Can be set
    t.daemon # Returns True if initialized as a daemon Thread. Can be set before the Thread has started.

    t.ident       # Returns a unique number to identify the Thread (on Linux, the pthread_t value)
    t.native_id   # Returns the OS-level thread-id (on Linux the tid of the Thread) 
```

#### Obsolete attributes / methods:
  ```
     t.getName()
     t.setName()
     t.isDaemon()
     t.setDaemon()
  ```

In [ ]:
##### Creating a new ```Thread``` class (OO-style threading)

from threading import Thread
class Counter(Thread):
    def __init__(self, count, delay, *args, **kwargs):
        super().__init__(*args, **kwargs)  # Python 3.3+
        # super(Thread, self).__init__(*args, **kwargs) # Python 3.0 to 3.2
        # Thread.__init__(self, *args, **kwargs)
        self.count = count
        self.delay = delay

    def run(self):
        from time import sleep
        for i in range(self.count):
            print(f"{self.name}: counting {i}")
            sleep(self.delay)

if __name__ == '__main__':
    c1 = Counter(count=5, delay=2, name="Count5")
    c2 = Counter(count=10, delay=1, name="Count10")

    c1.start()
    c2.start()

    c1.join()
    c2.join()
    

In [ ]:
##### Creating a new ```Thread``` to run python function

from threading import Thread, current_thread as current

def counter(count, delay):
    from time import sleep
    t = current()  # Returns the thread instance currently running this function
    for i in range(count):
        print(f"{t.name}: counting {i}")
        sleep(delay)

if __name__ == '__main__':
    c1 = Thread(target=counter, name="Count5", args=(5, 2))
    c2 = Thread(target=counter, name="Count10", kwargs={"count": 10, "delay": 1})

    c1.start()
    c2.start()

    c1.join()
    c2.join()
    

##### Active Threads vs Daemon Threads
 - Active Threads keep the python process active. Python process exits when all active threads terminate. By default the main thread of the python process is an active thread, so are threads created and started using ```Thread``` instances when their daemon attribute is not set.

 - Daemon Threads are automatically terminated when there are no more Active Threads alive in a python process. A daemon thread can be created by setting the ```daemon=True``` argument while constructing the Thread class, or setting the ```.daemon``` attribute to True to a Thread instance before the thread's ```.start()``` method is invoked.

##### Common use-cases for Daemon Threads
 - Heart-beat monitoring threads
 - Housekeeping / routine cleanup threads
 - Most daemon threads are designed to run an infinite loop that waits for an event or timed sleep, wakes up and perform a routine activities

##### Important notes
 1. Caution must be exercised if a daemon thread should be use resources managed by active threads. For example, reading from files which could be closed by active threads can lead to runtime errors in daemon threads that could not be gracefully handled.
 2. It is pointless to ```join()``` on a daemon thread as that will keep the active thread alive and waiting until the daemon thread exits.

#### Important helper functions in ```threading``` module:
```
    threading.current_thread()    # Returns the instance of current Thread
    threading.active_count()      # Returns the number of threads alive
    threading.enumerate()         # Returns the list of thread instances which are alive
    threading.main_thread()       # Returns the instance of the main thread
```